In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import torch
from spectral_hyperresolution.linear_reassignment import high_resolution_spectogram as high_resolution_spectogram_numpy
from spectral_hyperresolution.linear_reassignment import high_resolution_spectogram_sparse as high_resolution_spectogram_numpy_sparse
from spectral_hyperresolution.linear_reassignment_pytorch import high_resolution_spectogram as high_resolution_spectogram_pytorch
from spectral_hyperresolution.linear_reassignment_pytorch_vectorized import high_resolution_spectogram as high_resolution_spectogram_vectorized
from scipy.io import wavfile
import librosa
from scipy.io import loadmat

In [2]:
result_matlab = loadmat('data/verify_correctness.mat')['result']
x_matlab = loadmat('data/verify_correctness.mat')['x']

In [4]:
q = 2;
tdeci = 450;
over = 20;
noct = 12;
minf = 4e-3;
maxf = 0.45;

Let us first run the calculations on the GPU using Pytorch.

In [6]:
%%time

result_pytorch_float64 = high_resolution_spectogram_pytorch(x_matlab, q, tdeci, over, noct, minf, maxf, torch.device('cuda'), dtype=torch.float64)

CPU times: user 8.6 s, sys: 2.8 s, total: 11.4 s
Wall time: 11.4 s


Using single precision instead of double precision is 2x faster but is slightly less accurate (the original MATLAB implementation uses double precision floats).

In [7]:
%%time

result_pytorch_float32 = high_resolution_spectogram_pytorch(x_matlab, q, tdeci, over, noct, minf, maxf, torch.device('cuda'), dtype=torch.float32)

CPU times: user 6.21 s, sys: 1.05 s, total: 7.26 s
Wall time: 6.28 s


There is also a vectorized implementation. The amount of data it can run on is limited by the available GPU RAM. This can be to some extent mitigated by the `chunks` parameter.

On a 1080ti and given the amount of data I was only able to run the vectorized version of the algorithm using `float32`. Still, on smaller amounts of data, it can be faster than the non-vectorized version. In one experiment I observed ten fold improvement in execution time.

In [8]:
%%time

result_pytorch_vectorized = high_resolution_spectogram_vectorized(x_matlab, q, tdeci, over, noct, minf, maxf, torch.device('cuda'), chunks=15, dtype=torch.float32)

CPU times: user 4.7 s, sys: 1.86 s, total: 6.56 s
Wall time: 5.58 s


Numpy running on the CPU is much, much slower. Still, it is ~20% faster than Pytorch code executed on the CPU.

Both the numpy code below and Pytorch implementations using `DoubleTensors` return exactly the same results at the Matlab code.

In [9]:
%%time

result_python_dense = high_resolution_spectogram_numpy(x_matlab, q, tdeci, over, noct, minf, maxf)

CPU times: user 33min 24s, sys: 1min 22s, total: 34min 47s
Wall time: 5min 48s


The sparse implementation is much slower. Part of the slow down is due to the fact that I had to resolve to using swap. Nonetheless, even on simpler spectograms, this implementation will be at least 2x slower than the one above.

The sparse implementation mimics most closely the Matlab implementation. It's memory footprint will grow (it's maximum size can be controlled by the `MAXL` parameter hard coded in the function body).

In [10]:
%%time

result_python_sparse = high_resolution_spectogram_numpy_sparse(x_matlab, q, tdeci, over, noct, minf, maxf)

CPU times: user 1h 24min 44s, sys: 1h 5min 54s, total: 2h 30min 38s
Wall time: 26min 55s


In [11]:
result_matlab.todense()[0, :10]

matrix([[7.84406079e-06, 2.43280844e-05, 2.38775212e-05, 3.25661946e-05,
         3.97327638e-05, 4.30144901e-05, 5.24843663e-05, 5.37024620e-05,
         5.86939640e-05, 9.62354563e-05]])

In [9]:
result_pytorch_float64[0, :10]

tensor([7.8441e-06, 2.4328e-05, 2.3878e-05, 3.2566e-05, 3.9733e-05, 4.3014e-05,
        5.2484e-05, 5.3702e-05, 5.8694e-05, 9.6235e-05], device='cuda:0',
       dtype=torch.float64)

In [10]:
result_pytorch_float32[0, :10]

tensor([7.8441e-06, 2.4328e-05, 2.3877e-05, 3.2566e-05, 3.9733e-05, 4.3015e-05,
        5.2484e-05, 5.3703e-05, 5.8694e-05, 9.6236e-05], device='cuda:0')

In [11]:
result_pytorch_vectorized[0, :10]

tensor([7.8440e-06, 2.4328e-05, 2.3877e-05, 3.2566e-05, 3.9733e-05, 4.3015e-05,
        5.2484e-05, 5.3703e-05, 5.8694e-05, 9.6235e-05], device='cuda:0')

In [12]:
result_python_dense[0, :10]

array([7.84406079e-06, 2.43280844e-05, 2.38775212e-05, 3.25661946e-05,
       3.97327638e-05, 4.30144901e-05, 5.24843663e-05, 5.37024620e-05,
       5.86939640e-05, 9.62354563e-05])

In [13]:
result_python_sparse.todense()[0, :10]

matrix([[7.84406079e-06, 2.43280844e-05, 2.38775212e-05, 3.25661946e-05,
         3.97327638e-05, 4.30144901e-05, 5.24843663e-05, 5.37024620e-05,
         5.86939640e-05, 9.62354563e-05]])

In [12]:
np.max(result_matlab.todense() - result_pytorch_float64.cpu().numpy())

1.2656542480726785e-14

In [13]:
np.max(result_matlab.todense() - result_pytorch_float32.cpu().numpy())

9.047337450041049e-05

In [14]:
np.max(result_matlab.todense() - result_pytorch_vectorized.cpu().numpy())

8.922167696012728e-05

In [16]:
np.max(result_matlab.todense() - result_python_dense)

1.199040866595169e-14

In [17]:
np.max(result_matlab.todense() - result_python_sparse.todense())

8.881784197001252e-15

In [20]:
np.allclose(result_matlab.todense(), result_python_dense, atol=1e-100)

True

In [21]:
np.allclose(result_matlab.todense(), result_python_sparse.todense(), atol=1e-100)

True

In [15]:
np.allclose(result_matlab.todense(), result_pytorch_float64.cpu().numpy(), atol=1e-100)

True

In [16]:
np.allclose(result_matlab.todense(), result_pytorch_float32.cpu().numpy(), atol=1e-4)

True

In [17]:
np.allclose(result_matlab.todense(), result_pytorch_vectorized.cpu().numpy(), atol=1e-4)

True